In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
path = "C:\\Dataframes\\Oracle\\files" # your path with double backslashes 
all_files = glob.glob(path + "/*.csv")

li = [pd.read_csv(filename, index_col=None, header=0) for filename in all_files]

df = pd.concat(li, axis=0, ignore_index=True)

In [5]:
df.shape

(1508507, 32)

In [4]:
df.head(3)

,session_id,date,time,screen_id,theatre_id,movie_id,facility_type,facility_location_type,latitude,longitude,...,featured,run_time,genres,actress,actor,director,music_director,music_label,release_type,sold_seats
0,4dc73062-49da-4956-a4ea-4ae6e6d129f5,2016-11-03,2016-11-03T18:30:00.000+05:30,c0287c52-c993-4cc6-9225-86eb16579b80,01668bc7-8262-4b03-88ff-4170a8811995,34324eda-2253-4ba9-bf19-9a7385f16e34,Complex,Standalone,18.410869,84.037351,...,False,7920,"['Action', 'Drama']",['Aditi Arya'],['Nandamuri Kalyan Ram'],['Puri Jagannadh'],['Anoop Rubens'],['Aditya Music'],Wide Release,0
1,d84ada4e-92e8-4efc-960b-bdde55185595,2016-11-03,2016-11-03T14:30:00.000+05:30,c0287c52-c993-4cc6-9225-86eb16579b80,01668bc7-8262-4b03-88ff-4170a8811995,34324eda-2253-4ba9-bf19-9a7385f16e34,Complex,Standalone,18.410869,84.037351,...,False,7920,"['Action', 'Drama']",['Aditi Arya'],['Nandamuri Kalyan Ram'],['Puri Jagannadh'],['Anoop Rubens'],['Aditya Music'],Wide Release,0
2,ea7cd215-93de-424a-8540-6235cf898c77,2016-11-03,2016-11-03T21:30:00.000+05:30,c0287c52-c993-4cc6-9225-86eb16579b80,01668bc7-8262-4b03-88ff-4170a8811995,34324eda-2253-4ba9-bf19-9a7385f16e34,Complex,Standalone,18.410869,84.037351,...,False,7920,"['Action', 'Drama']",['Aditi Arya'],['Nandamuri Kalyan Ram'],['Puri Jagannadh'],['Anoop Rubens'],['Aditya Music'],Wide Release,0


In [5]:
to_drop = ['session_id', 'latitude', 'longitude', 'area', 'cooling_type', 'featured', 'release_type']
for col in to_drop:
    df.drop(col, axis=1, inplace=True)

In [9]:
def get(people):
    return people.strip('[').strip(']').replace("'", "")

for col in df.columns[18:24]:
    df[col] = df[col].apply(get)

In [10]:
from datetime import date

df.drop('date',axis=1,inplace=True)

def weekend(dt):
    ymd = dt.split('T')[0].split('-')
    y, m, d = ymd[0], ymd[1].lstrip('0'), ymd[2].lstrip('0')
    y, m, d = int(y), int(m), int(d)
    if date(y, m, d).weekday() > 3:
        return 1
    else:
        return 0

df['weekend'] = df['time'].apply(weekend)

In [11]:
def time_of_day(time):
    hour = time.split('T')[1][:2]
    if hour[0] == '0':
        hour = int(hour[1])
    else:
        hour = int(hour)
    
    if hour < 13:
        return 'Morning'
    elif hour < 17:
        return 'Afternoon'
    elif hour < 20:
        return 'Evening'
    elif hour <= 23:
        return 'Night'

df['time_of_day'] = df['time'].apply(time_of_day)

In [18]:
df['certificate'] = df['certificate'].apply(lambda x: 'U/A' if pd.isna(x) else x)

df['total_seats'] = df['total_seats'].apply(int)

df = df[np.isfinite(df['total_seats'])]
df = df[pd.notnull(df['release_date'])]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
import datetime

df['show_date'] = df['time'].apply(lambda x: x.split('T')[0])

def get_date(d):
    ymd = d.split('-')
    y, m, d = ymd[0], ymd[1].lstrip('0'), ymd[2].lstrip('0')
    y, m, d = int(y), int(m), int(d)
    return date(y, m, d)

df['show_date'] = df['show_date'].apply(get_date)
df['release_date'] = df['release_date'].apply(get_date)

df['since_release'] = [(df['show_date'][i] - df['release_date'][i]).days for i in range(len(df))]

In [116]:
df.columns

Index(['time', 'screen_id', 'theatre_id', 'movie_id', 'facility_type',
       'facility_location_type', 'city', 'province', 'postal_code',
       'total_seats', 'projection_type', 'sound_mixes', 'country',
       'release_date', 'certificate', 'language', 'run_time', 'genres',
       'actress', 'actor', 'director', 'music_director', 'music_label',
       'sold_seats', 'weekend', 'time_of_day', 'show_date', 'since_release'],
      dtype='object')

In [121]:
for feature in df.columns:
    print(feature + ': ' + str(len(df[feature].unique())))

time: 105407
screen_id: 760
theatre_id: 412
movie_id: 2795
facility_type: 3
facility_location_type: 2
city: 243
province: 22
postal_code: 330
total_seats: 309
projection_type: 4
sound_mixes: 30
country: 1
release_date: 488
certificate: 3
language: 22
run_time: 118
genres: 249
actress: 1777
actor: 1901
director: 2175
music_director: 1574
music_label: 459
sold_seats: 984
weekend: 2
time_of_day: 4
show_date: 724
since_release: 2156


In [ ]:
# screen, theatre, movie, facility_type, genre, stars, weekend, time_of_day